<a href="https://colab.research.google.com/github/ShivaKondapalli/NLPColabNotebooks/blob/master/Classification_with_Recurrent_Neural_Networks_RNN_and_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learnng for Natural Language Processing - I


## Introduction 

In this Notebook, we classify the names of people to thier nationality.

The code reference for this notebook is [Names to Nationality PyTorch](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

Further,  two reucurrent models: *Vanilla RNNs'* and the* Gated Reucrrent Unit * are trained on the same data and a comparison  of their * Confusion matriices* is performed. 

Let us start off by delineating how RNN's differ from fully connected or convolutional models and how a gru network is a variation on the vanilla Rnn that was created to address some of it's limitations. 

# Recurrent Neural Networks

## Fully Connected Neural Network

A fully connected Neural Network passes an n-dimentional input vector through a series of *Linear transformations*
followed by a pointwise *Non-Linearity*, could be ReLU, Tanh or Sigmoid. Their is no feeding of any output back into the 
input. 

Here is a picture of a fully connected Neural Network. 

![Image source](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS6D4knBcTXoWbJgrngE_VcFqK6gsfnrssYnAs7Ts-Ee9nNDCSJ)

The above pircture is from the paper: [Tropical Geometry of Deep Neural Networks](https://arxiv.org/abs/1805.07091)

The above paper is highly recommended. Reading it currently and hope to write about it soon. 

A recurrent netwokr operates on a sequences of data as opposed to a single instance. 

Ex: Happy. 

The text example given above is a word, which can be interpreted as a sequence of characters. The RNN operates on each character of this string. Each input is called a "*timestep*". 

Now, the structure of a languge is such that it is compositional. 
Every successive character in a word, or every successive word in a sentence is dependent on the previous character or word. 

Recurrent neural neworks operate on a  sequence of such inputs and pass each previous input to the next time step.

**Time step 1**:  "H" as the input to our network - timestep 1

**Time step 2** : "a" along with the previous input H is what the second time step recieves. This is called the *hidden state*  of the network.

**Time step 2**: This one gets "a" from the previous state and "p" the current input


A picture will clear things up. 

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-rolled.png)

Image Source: [Chirs Olah](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

This is a *rolled* version of an RNN, at each time step t, it gets an input $\ {x_t}$ and  the output of the previous hidden state $\ {h_{t-1}}$.  

A linear mapping of the current input and previous hidden state is passed into a hyperbloc tangent function to predict the current hidden state  $\ {h_t}$.

$\ {h_t = \tanh(W_x* x_t + b_x + W_h* h_{t-1} + b_h)}$

This hidden state is propogated to the next time step. This gets combined with the current input at that time step to predict the current hidden state for that time step.

An *unrolled* version can make things clearer. 

![alt text](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png)

Image Source: [Chirs Olah](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

1. A the first time step, $\ {h_0}$ & $\ {x_0}$ are initialized to zero. 

2. The output from this unit is propogated to he the next time step. 

3. This time step gets the current input, i.e. the first character "H" and the previous hidden state.

4. The next time step takes the current chracter "a"  and the previous hidden state for the letter '"H"'.

5. This is propogated till the very last character of the sequence is exhausted. 


Since we are preforming classification. We take the last Hidden output and pass it through a linear layer and then a softmax to get a probability distribution over the characters. 

## Limitations of Vanilla RNN's.

Language is a tricky thing. Let us take the following example into consideration. 

'"He was a really thin and pale, had no strength at all, and for this reason the doctor conducted a number 
of tests on him. "

The last word "him" in the sentence is dependent on the very first word He, these are called "Long range dependcies", where a word in sentence exists because of some word at the beginning of the sentence. 

A vanilla rnn has information only from the previous time step, if it were operating on long sequence of data, it is very likely to forget what it saw earlier. We thus need some regulatory mechanism to remeber information in our sentences so that they  can be passed on to the next time step or forgotten if they are not important. 

Tow variations on RNN's have been devised for this. In this notebook we talk of the first. 

## Gated Recurrent Unit.

The Gated Recurrent Unit is one such variation of a recurent neural network.

In this network, we have contsructs called "*gates*" which control the flow of information. A GRU has three gates 

1. Reset : $\ {r_t = \sigma(W_{ir} * x_t + b_{ir} + W_{hr} * h_{t-1} + b_{hr})}$
2. Update : $\ {u_t  = \sigma(W_{iu} * x_t + b_{iu} + W_{hu} * h_{t-1} + b_{hu})}$
3. New gate : $\ {n_t = \tanh(W_{in} * x_t + b_{in} + r_t * W_{hn} * h_{t-1} + b_{hn})}$

Hidden State: $\ {h_t = (1- u_t) * n_t + u_t * h_{t-1}}$

What is this equations tellin us? Well the Update gate and reset gate both have sigmoid functions. 

This means thier values lie between 0 and 1.

The reset gate in equation 2 mesaures the importance of the previous hidden step in our new gate. 

If the update gate is close to 1, the new gate will get closer to zero. This is telling the network to retain the information from the previous hidden state.

This is: $\ {h_t = u_t * h_{t-1}}$ sicne $\ {u_t \approx 0}$ the hidden state value is retained, 

If on the other hand, the new gate was close to zero, the information woule be overwridden with the new hidden state.

Now, of course the gate values are not strictly binary, but thinking about them as such gives us a conceptual handle on how information gets apassed in our network.

This is precisely what allows for regulating information flow in a GRU. This inturn allows for long range dependecies. 
A more powerful model called the LSTM model, has more gates and is even better for modelling long range dependencies. 

We shall talk about them in a later post. 






In [0]:
# All imports

import torch
import torch.nn as nn
import unicodedata
import string
from io import open
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import glob

In [7]:
# Check for GPU

if torch.cuda.is_available():
  print('The power of the GPU is with oyou')
else:
  print('Do  not bother running this')

The power of the GPU is with oyou


In [0]:
# get path and extension

folder_path = "sample_data"
ext = "*.txt"

'sample_data/Arabic.txt'

def get_files(path):
    return glob.glob(os.path.join(folder_path, ext))


all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)


def unicodetoascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn' and c in all_letters)


# Build the category_lines dictionary, a list of names per language
category_names = {}
all_categories = []


# Read a file and split into lines
def readfiles(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodetoascii(line) for line in lines]

# loop through the files.                      
for f in get_files(folder_path):
  category = f.split('/')[1].split('.')[0]
  all_categories.append(category)
  lines = readfiles(f)
  category_names[category] = lines
  
n_categories = len(all_categories)

## Text Pre-Processing 

We need to conert our tensors to one-hot vectors.
This is because neural networks can't take characters directly as input. They need numbers. 

One hot vectors aren't the best representation for text, they are sparse and don't really capture any semantic similarity 
between words.

**Word Embeddings** are a much better representation for text as they are dense, i.e. each dimension of the vector captures a certain quantiative measure of the word we are interested in. 

They are a little out of scope for this post at the moment, but will surely be convered later.

In [0]:
# converting names to tensors
# Here we do some pre-processing to convert names to tenosrs

def lettertoindex(l):
    """converts letter to index"""
    return all_letters.index(l)


def lettertotensor(l):
    """converts a letter to a tensor"""
    tensor = torch.zeros(1, len(all_letters))
    l_idx = lettertoindex(l)
    tensor[0][l_idx] = 1
    return tensor

def nametotensor(name):
    """converts a name into a tensor of shape seq, 1, len(all_letters)"""
    tensor = torch.zeros(len(name), 1, len(all_letters))
    for idx, l in enumerate(name):
        tensor[idx][0][lettertoindex(l)] =1
    return tensor


def categoryfromoutput(output):
    top_v, top_i = output.topk(1)
    cat_i = top_i[0].item()
    return all_categories[cat_i], cat_i

## Two Models: RNN & GRU

In [0]:
# Here is the class of our RNN network

class RNN(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):

        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        combined = torch.cat((x, hidden), dim=1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def inithidden(self):
        return torch.zeros(1, self.hidden_size)


class GRU(nn.Module):

    def __init__(self, input_size, hidden_size, output_size, num_layers=1):

        super(GRU, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers =num_layers

        self.gru  = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):

        batch_size = x.size(1)

        hidden = self.inithidden(batch_size)
        output, hidden = self.gru(x, hidden)
        fc_out = self.fc(hidden)
        return fc_out

    def inithidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)

# instantiating our data 

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)
gru = GRU(n_letters, n_hidden, n_categories)

In [0]:
# defined random choice to get a random number of examples for our data
def random_choice(lst):
    return lst[np.random.randint(0, len(lst)-1)]


def randomtrainningexample():
    category = random_choice(all_categories)
    name = random_choice(category_names[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    name_tensor = nametotensor(name)
    return category, name, category_tensor, name_tensor
 

## Training 

We will train a plain vanilla RNN and  A GRU network

In [0]:
# Setting hyperparameters for our data 

# setting hyperparameters
learning_rate = 0.005
criterion_rnn = nn.NLLLoss()
criterion_gru = nn.CrossEntropyLoss()

def train_rnn(category_tensor, name_tensor):
    hidden = rnn.inithidden()

    rnn.zero_grad()

    for i in range(name_tensor.size()[0]):
        output, hidden = rnn.forward(name_tensor[i], hidden)

    loss = criterion_rnn(output, category_tensor)
    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)  # can also use torch.optim() if you so choose to

    return output, loss.item()


def train_gru(category_tensor, name_tensor):

    gru.zero_grad()

    output = gru.forward(name_tensor)

    loss = criterion_gru(output.squeeze(1), category_tensor)
    loss.backward()

    for p in gru.parameters():
        p.data.add_(-learning_rate, p.grad.data)  # can also use torch.optim() if you so choose to

    return output, loss.item()


def time_taken(start):
    time_elapsed = time.time() - start
    min = time_elapsed//60
    sec = time_elapsed%60
    return '%dm %ds' % (min, sec)


def evaluate(name_tensor, model):

    if model == rnn:

        hidden = rnn.inithidden()

        for i in range(name_tensor.size()[0]):
            output, next_hidden = rnn.forward(name_tensor[i], hidden)

    elif model == gru:

        output = gru.forward(name_tensor)

    return output


def predict(name, model, n_predictions=3):
    print(name)

    with torch.no_grad():
        output = evaluate(nametotensor(name), model)

        if model == gru:
            output = output.squeeze(1)

        top_n, top_i = output.topk(n_predictions, 1, True)
        predictions_lst = []

        for i in range(n_predictions):
            val = torch.exp(top_n[0][i]) if model == rnn else top_n[0][i]
            cat_idx = top_i[0][i].item()
            print(f'Value: {val.item()}, language: {all_categories[cat_idx]}')
            predictions_lst.append([val, all_categories[cat_idx]])



# # Confusion Matrix :

We plot confusion matrices for both the network we train. 

The principal diagonal of the matrix shows us the correct predictions. The brighter the digonal the better the model. 


In [0]:
# Training
n_iters = 100000
print_every = 5000
plot_every = 1000
    
current_loss_rnn = 0
all_losses_rnn = []

start = time.time()

print('Training Vanilla RNN')
print(' ')

# Training Vanilla RNN
for i in range(1, n_iters+1):
  
  category, name, category_tensor, name_tensor = randomtrainningexample()
  output, loss = train_rnn(category_tensor, name_tensor)
  
  current_loss_rnn += loss
  
  if n_iters % print_every == 0:
    pred, pred_i = categoryfromoutput(output)
    prediction = 'True' if pred == category else f'False, correct one is {category}'
    
    print('%d %d%% (%s) %.4f %s / %s %s' % (i, i / n_iters * 100, time_taken(start), loss, name, pred, prediction))
    
  if i % plot_every == 0:
    all_losses_rnn.append(current_loss_rnn/plot_every)
    current_loss_rnn = 0

print('')
print('############################################################################')
print('Training GRU network now')

# Training Gated Recurrent Unit
current_loss_gru = 0
all_losses_gru = []

for i in range(1, n_iters+1):
  category, name, category_tensor, name_tensor = randomtrainningexample()
  output, loss = train_gru(category_tensor, name_tensor)
  current_loss_gru += loss
  
  if n_iters % print_every == 0:
    pred, pred_i = categoryfromoutput(output)
    prediction = 'True' if pred == category else f'False, correct one is {category}'
    print('%d %d%% (%s) %.4f %s / %s %s' % (i, i / n_iters * 100, time_taken(start), loss, name, pred, prediction))
    
  if i % plot_every == 0:
    all_losses_gru.append(current_loss_gru/plot_every)
    current_loss_gru = 0

# plot confusion matrix and losses
confusion_rnn = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Add one to each row: the real category. Each column: the predicted category.
# The darker the principal diagonal, the better the model.

for i in range(n_confusion):
  category, name, category_tensor, name_tensor = randomtrainningexample()
  output_rnn = evaluate(name_tensor, rnn)
  guess, guess_i_rnn = categoryfromoutput(output_rnn)
  real_category_i = all_categories.index(category)
  confusion_rnn[real_category_i][guess_i_rnn] += 1

for i in range(n_categories):
  confusion_rnn[i] = confusion_rnn[i]/confusion_rnn[i].sum()

# Set up fig, axes.

fig = plt.figure()
ax1 = fig.add_subplot(221)
cax = ax1.matshow(confusion_rnn.numpy())
fig.colorbar(cax)

# Set the labels for x and y axes
ax1.set_xticklabels([''] + all_categories, rotation=90)
ax1.set_yticklabels([''] + all_categories)

# Major tick locations on the axis are set.
ax1.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax1.yaxis.set_major_locator(ticker.MultipleLocator(1))

# Plot Vanilla Rnn losses
ax1 = fig.add_subplot(222)
ax1.set_title('Vanilla Rnn Losses')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Losses')
ax1.plot(all_losses_rnn)

# Gated Recurrent unit
confusion_gru = torch.zeros(n_categories, n_categories)
n_confusion = 10000

# Add one to each row: the real category and each column: the predicted category.
# The darker the principal diagonal, the better the model.

for i in range(n_confusion):
  category, name, category_tensor, name_tensor = randomtrainningexample()
  output_gru = evaluate(name_tensor, gru)
  guess, guess_i_gru = categoryfromoutput(output_gru)
  real_category_i = all_categories.index(category)
  confusion_gru[real_category_i][guess_i_gru] += 1

for i in range(n_categories):
  confusion_gru[i] = confusion_gru[i]/confusion_gru[i].sum()

ax1 = fig.add_subplot(223)
cax1 = ax1.matshow(confusion_gru.numpy())
fig.colorbar(cax1)

# Set the labels for x and y axes
ax1.set_xticklabels([''] + all_categories, rotation=90)
ax1.set_yticklabels([''] + all_categories)

# Major tick locations on the axis are set.
ax1.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax1.yaxis.set_major_locator(ticker.MultipleLocator(1))

# Plot GRU losses
ax1 = fig.add_subplot(224)
ax1.set_title('GRU losses')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Losses')
ax1.plot(all_losses_gru)

plt.show()

   

In [0]:
# predict for vanilla Rnn
predict('Akutagawa', rnn)
predict('Avgerinos', rnn)
predict('Lestrange', rnn)

# predict for GRU
predict('Akutagawa', gru)
predict('Avgerinos', gru)
predict('Lestrange', gru)